In [1]:
from tasho.templates.P2P import P2P
from tasho import TaskModel, default_mpc_options
from tasho.templates.Regularization import Regularization
from tasho.Variable import Variable
from tasho.ConstraintExpression import ConstraintExpression
from tasho.Expression import Expression
from tasho.MPC import MPC 
from tasho import environment as env
from tasho import WorldSimulator
from tasho.OCPGenerator import OCPGenerator
from robotshyu import Robot as rob
from time import sleep
import numpy as np
import casadi as cs
import pybullet as p
import tasho
import robotshyu
import os

pybullet build time: May 20 2022 19:44:17


In [2]:
# Options for example codes
frame_enable = True

# Ex1) P2P example

In [3]:
robot = rob.Robot("indy7")
link_name = 6
goal_pose = Variable(robot.name, "goal_pose", "magic_number", (4,4), np.array(
        [[0, 0, 1, 1], [0, 1, 0, -0.1], [-1, 0, 0, 0.6], [0, 0, 0, 1]]
    ))
q0 = [1.0, -0.05311582280659044, -0.815452580946695, -1.3191046402052224, -0.8582660722530533, 1.3988994390898029]

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)
ndof = robot.nq
q_current = Variable(robot.name, "jointpos_init", 'magic_number', (ndof, 1), q0)

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [4]:
task_P2P = P2P(robot, link_name, goal_pose, q_current, 0.001)

In [5]:
#task_P2P.write_task_graph("Indy7_P2P_Task.svg")

In [6]:
task_P2P.add_path_constraints(Regularization(task_P2P.variables['qd_'+robot.name], 1),
                              Regularization(task_P2P.variables['qdd_'+robot.name], 0.1))

In [7]:
#task_P2P.write_task_graph("Indy7_P2P_Task2.svg")

In [8]:
horizon_steps = 10
horizon_period = 3.0
q_pred = [0]*horizon_steps

OCP_gen = OCPGenerator(task_P2P, False, {"time_period": horizon_period, "horizon_steps":horizon_steps})
q_ocp = OCP_gen.stage_tasks[0].variables['q_'+robot.name].x
OCP_gen.tc.set_initial(q_ocp, q0)
OCP_gen.tc.set_ocp_solver(
    "ipopt", 
    # {"ipopt":{"linear_solver":"ma27"}}
)
OCP_gen.tc.solve_ocp()


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.7, running with linear solver ma27.

Number of nonzeros in equality constraint Jacobian...:      452
Number of nonzeros in inequality constraint Jacobian.:      186
Number of nonzeros in Lagrangian Hessian.............:      141

Total number of variables............................:      192
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:      141
Total number of inequality constrai

In [9]:
t_grid, qsol = OCP_gen.tc.sol_sample(q_ocp)
_, q_dot_sol = OCP_gen.tc.sol_sample(OCP_gen.stage_tasks[0].variables['qd_'+robot.name].x)

obj = WorldSimulator.WorldSimulator()

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position,orientation,robot_pkg=robot)
frameID0 = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)
robotIDs=[robotID,frameID0]

if frame_enable:
    frameIDs = [0]*horizon_steps
    for i in range(horizon_steps):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)


for i in range(horizon_steps):
    q_pred[i]=qsol[i]

joint_indices = [0, 1, 2, 3, 4, 5]

obj.resetMultiJointState(robotIDs, joint_indices, [q0])

if frame_enable:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0])
    obj.resetMultiJointState(frameIDs,joint_indices,q_pred)

In [10]:
for i in range(horizon_steps + 1):
    sleep(horizon_period*0.5/horizon_steps)
    obj.resetMultiJointState(
        robotIDs, joint_indices, [qsol[i]]
    )
    
sleep(0.5)
obj.end_simulation()

Ending simulation


# Ex2) Object picking MPC

In [11]:
robot = rob.Robot("indy7")
link_name = 6

# Define initial conditions of the robot
q0_val = [0, -0.423598, -0.9, 0.0, -1.5709, -0.523598]
qd0_val = [0] * robot.ndof

robot.set_joint_acceleration_limits(lb=-360*3.14159/180, ub=360*3.14159/180)

################################################
# Task spacification - Approximation to object
################################################

# Select prediction horizon and sample time for the MPC execution
horizon_size = 10
t_mpc = 0.05

q_init = Variable(robot.name, 'q_init', 'parameter', (robot.ndof,1))
qd_init = Variable(robot.name, 'qd_init', 'parameter', (robot.ndof,1))
goal_pose = Variable(robot.name, "goal_pose", 'parameter', (4,4))

task_P2P = P2P(robot, link_name, goal_pose, q_init, 0.001)
#task_P2P.write_task_graph("Indy7_P2P_Task4.svg")

Loading robot params from json ...
Loaded 6-DoF robot: indy7


In [12]:
task_P2P.remove_initial_constraints(task_P2P.constraint_expressions['stationary_qd_indy7'])
task_P2P.remove_terminal_constraints('rot_con_pose_6_indy7_vs_goal',
                                    'trans_con_pose_6_indy7_vs_goal')
#task_P2P.write_task_graph("Indy7_P2P_Task5.svg")

In [13]:
task_P2P.add_initial_constraints(ConstraintExpression(robot.name, "eq", Expression(robot.name, "err_qd_qinit", lambda a, b : a - b, qd_init, task_P2P.variables['qd_indy7']),
                             "hard", reference = 0))
task_P2P.add_path_constraints(Regularization(task_P2P.expressions['trans_error_pose_6_indy7_vs_goal'], 5e0, norm = "L1"),
                            Regularization(task_P2P.expressions['ax_ang_error_pose_6_indy7_vs_goal'], 1e0), 
                            Regularization(task_P2P.variables['qd_indy7'], 1e-1),
                            Regularization(task_P2P.variables['qdd_indy7'], 1e-3))

#task_P2P.write_task_graph("Indy7_P2P_Task6.svg")

In [14]:
pOCP = OCPGenerator(task_P2P, False, {"time_period":horizon_size*t_mpc, "horizon_steps":horizon_size})

tc = pOCP.tc

################################################
# Set solver and discretization options
################################################
mpc_options = default_mpc_options.get_default_mpc_options()

tc.ocp_solver = mpc_options["ipopt_init"]["solver_name"]
tc.ocp_options = mpc_options["ipopt_init"]["options"]
tc.mpc_solver = mpc_options["ipopt_lbfgs"]["solver_name"]
tc.mpc_options =mpc_options["ipopt_lbfgs"]["options"]
tc.set_ocp_solver(tc.ocp_solver, tc.ocp_options)

################################################
# Set parameter values
################################################
q = pOCP.stage_tasks[0].variables['q_indy7'].x
qd = pOCP.stage_tasks[0].variables['qd_indy7'].x
q_0 = pOCP.stage_tasks[0].variables['q_init_indy7'].x
qd_0 = pOCP.stage_tasks[0].variables['qd_init_indy7'].x
goal_pose = pOCP.stage_tasks[0].variables['goal_pose_indy7'].x
goal_pose_val =  cs.vertcat(
    cs.hcat([-1, 0, 0, 0.5]),
    cs.hcat([0, 1, 0, 0.0]),
    cs.hcat([0, 0, -1, 0.25]),
    cs.hcat([0, 0, 0, 1]),
)
tc.set_initial(q, q0_val)
tc.set_value(goal_pose, goal_pose_val)
tc.set_value(q_0, q0_val)
tc.set_value(qd_0, qd0_val)

# Add an output port for joint velocities as well
tc.tc_dict["out_ports"].append({"name":"port_out_qd_indy7", "var":"qd_indy7", "desc": "output port for the joint velocities"})

# Add a monitor for termination criteria
tc.add_monitor({"name":"termination_criteria", "expression":cs.sqrt(cs.sumsqr(pOCP.stage_tasks[0].expressions["trans_error_pose_6_indy7_vs_goal"].x)) - 2e-2, "reference":0, "lower":True, "initial":True})

# os.system("export OMP_NUM_THREADS = 1")

sol = tc.solve_ocp()

startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context current
GL_VENDOR=Mesa/X.org
GL_RENDERER=llvmpipe (LLVM 12.0.0, 256 bits)
GL_VERSION=4.5 (Core Profile) Mesa 21.2.6
GL_SHADING_LANGUAGE_VERSION=4.50
pthread_getconcurrency()=0
Version = 4.5 (Core Profile) Mesa 21.2.6
Vendor = Mesa/X.org
Renderer = llvmpipe (LLVM 12.0.0, 256 bits)
b3Printf: Selected demo: Physics Server
startThreads creating 1 threads.
starting thread 0
started thread 0 
MotionThreadFunc thread started
ven = Mesa/X.org
ven = Mesa/X.org
numActiveThreads = 0
stopping threads
Thread with taskId 0 exiting
Thread TERMINATED
destroy semaphore
semaphore destroyed
destroy main semaphore
main semaphore destroy

In [15]:
dir_casadi_func = "casadi_dir"
os.makedirs("./"+dir_casadi_func, exist_ok=True)
vars_db = tc.generate_MPC_component("./"+dir_casadi_func+"/", {"ocp_cg_opts":{"save":True, "codegen":False, "jit":False}, "mpc":True, "mpc_cg_opts":{"save":True, "codegen":False, "jit":False}})

@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)
@1=(opti1_p_2[12:15]-fk_T(opti1_x_1){6}[12:15]), (sqrt(dot(@1, @1))-0.02)


In [16]:
MPC_component = MPC("indy7_obj_pickup", "./"+dir_casadi_func+"/"+ tc.name + ".json")

In [17]:
obj = WorldSimulator.WorldSimulator(bullet_gui=True)

# Add robot to the world environment
position = [0.0, 0.0, 0.0]
orientation = [0.0, 0.0, 0.0, 1.0]

robotID = obj.add_robot(position,orientation,robot_pkg=robot)

if frame_enable:
    frameIDs = [0]*horizon_size
    for i in range(horizon_size):
        frameIDs[i] = obj.add_robot(position,orientation,robot_pkg=robot,frame=True)

# Set environment
environment = env.Environment()
package_path = tasho.__path__[0]
cube1 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small.urdf")
environment.add_object(cube1, "cube")
table1 = env.Box(height = 0.3, position = [0.5, 0, 0], orientation = [0.0, 0.0, 0.7071080798594737, 0.7071054825112364], urdf =package_path+ "/models/objects/table.urdf")
environment.add_object(table1, "table1")

cube2 = env.Cube(length = 1, position = [0.5, -0.2, 0.35], orientation = [0.0, 0.0, 0.0, 1.0], urdf = package_path+"/models/objects/cube_small_col.urdf", fixed = True)
environment.add_object(cube2, "cube2")
environment.set_in_world_simulator(obj)
cubeID = environment.get_object_ID("cube")
cube2ID = environment.get_object_ID("cube2")

p.resetBaseVelocity(cubeID, linearVelocity=[0, 0.8, 0])
# Determine number of samples that the simulation should be executed
no_samples = int(t_mpc / obj.physics_ts)
if no_samples != t_mpc / obj.physics_ts:
    print("[ERROR] MPC sampling time not integer multiple of physics sampling time")

# Correspondence between joint numbers in bullet and OCP
joint_indices = [0, 1, 2, 3, 4, 5]

 # Begin the visualization by applying the initial control signal
ts, q_sol = tc.sol_sample(q, grid="control")
ts, q_dot_sol = tc.sol_sample(qd, grid="control")

obj.resetJointState(robotID,joint_indices,q0_val)
if frame_enable==True:
    obj.resetMultiJointState(frameIDs, joint_indices, [q0_val])

obj.setController(
    robotID, "velocity", joint_indices, targetVelocities=q_dot_sol[0]
)
q_log = []
q_dot_log = []
predicted_pos_log = []
q_pred = [0]*horizon_size


In [18]:
# Execute the MPC loop

for i in range(horizon_size * 30):
    print("----------- MPC execution -----------")

    q_now = obj.readJointPositions(robotID, joint_indices)
    qd_now = obj.readJointVelocities(robotID, joint_indices)
        
    MPC_component.input_ports["port_inp_q_init_indy7"]["val"] = q_now
    MPC_component.input_ports["port_inp_qd_init_indy7"]["val"] = qd_now

    # Predict the position of the target object (cube)
    lin_vel, ang_vel = p.getBaseVelocity(cubeID)
    lin_vel = cs.DM(lin_vel)
    lin_pos, _ = p.getBasePositionAndOrientation(cubeID)
    lin_pos = cs.DM(lin_pos)
    time_to_stop = cs.norm_1(lin_vel) / 0.3
    predicted_pos = (
        cs.DM(lin_pos)
        + cs.DM(lin_vel) * time_to_stop
        - 0.5 * 0.5 * lin_vel / (cs.norm_1(lin_vel) + 1e-3) * time_to_stop ** 2
    )
    predicted_pos_log.append(predicted_pos.full())
    p.resetBasePositionAndOrientation(cube2ID, predicted_pos.full(), [0.0, 0.0, 0.0, 1.0])
    predicted_pos[2] += 0.06  # cube height
    print("Predicted position of cube", predicted_pos)
    predicted_pos_val = cs.vertcat(
        cs.hcat([-1, 0, 0, predicted_pos[0]]),
        cs.hcat([0, 1, 0, predicted_pos[1]]),
        cs.hcat([0, 0, -1, predicted_pos[2]]),
        cs.hcat([0, 0, 0, 1]),
    )

    MPC_component.input_ports["port_inp_goal_pose_indy7"]["val"] = cs.vec(predicted_pos_val)
        
    if i == 0:
        MPC_component.configMPC()

    MPC_component.runMPC()
    if frame_enable==True:
        sol = MPC_component.res_vals
        for i in range(horizon_size):
            q_pred[i]=sol[robot.ndof*i:robot.ndof*i+robot.ndof].full()
        obj.resetMultiJointState(frameIDs, joint_indices, q_pred)

    q_log.append(q_now)
    q_dot_log.append(qd_now)

    # Set control signal to the simulated robot
    qd_control_sig = MPC_component.output_ports["port_out_qd_indy7"]["val"].full()
    qdd_control_sig = (MPC_component.output_ports["port_out_qdd_indy7"]["val"] * t_mpc).full()
    
    obj.setController(
        robotID, "velocity", joint_indices, targetVelocities=qd_control_sig+qdd_control_sig
    )
    
    # Simulate
    obj.run_simulation(no_samples)

    # Termination criteria
#     if "termination_criteria_true" in MPC_component.event_output_port:
#         break

----------- MPC execution -----------
Predicted position of cube [0.5, 0.15777500346789, 0.41]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   5.27ms (658.50us) 269.48us ( 33.69us)         8
       nlp_g  | 904.00us (113.00us)  72.82us (  9.10us)         8
    nlp_grad  |  85.00us ( 85.00us)  85.84us ( 85.84us)         1
  nlp_grad_f  |  11.59ms (  1.29ms) 739.80us ( 82.20us)         9
  nlp_hess_l  |  21.43ms (  3.06ms)   2.98ms (425.31us)         7
   nlp_jac_g  | 317.00us ( 35.22us) 126.92us ( 14.10us)         9
       total  |  88.11ms ( 88.11ms)   9.09ms (  9.09ms)         1
startThreads creating 1 threads.
starting thread 0
started thread 0 
argc=2
argv[0] = --unused
argv[1] = --start_demo_name=Physics Server
ExampleBrowserThreadFunc started
X11 functions dynamically loaded using dlopen/dlsym OK!
X11 functions dynamically loaded using dlopen/dlsym OK!
Creating context
Created GL 3.3 context
Direct GLX rendering context obtained
Making context c

----------- MPC execution -----------
Predicted position of cube [0.499999682618, 0.31677327478954, 0.2724911803854]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 221.00us ( 31.57us) 221.49us ( 31.64us)         7
       nlp_g  |  57.00us (  8.14us)  55.73us (  7.96us)         7
    nlp_grad  | 105.00us (105.00us) 106.06us (106.06us)         1
  nlp_grad_f  | 643.00us ( 80.38us) 646.97us ( 80.87us)         8
  nlp_hess_l  |  25.90ms (  4.32ms)   2.53ms (421.74us)         6
   nlp_jac_g  |   5.80ms (725.13us) 134.95us ( 16.87us)         8
       total  |  47.63ms ( 47.63ms)   7.28ms (  7.28ms)         1
----------- MPC execution -----------
Predicted position of cube [0.4999996762801, 0.32652383191909, 0.27249040567551]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 216.00us ( 30.86us) 215.40us ( 30.77us)         7
       nlp_g  |  52.00us (  7.43us)  52.64us (  7.52us)         7
    nlp_grad  | 103.00us (103.00us) 10

----------- MPC execution -----------
Predicted position of cube [0.49999809082555, 0.43299045436011, 0.084989297598358]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 186.00us ( 31.00us) 185.43us ( 30.91us)         6
       nlp_g  |  47.00us (  7.83us)  45.74us (  7.62us)         6
    nlp_grad  |  99.00us ( 99.00us)  99.46us ( 99.46us)         1
  nlp_grad_f  |   8.53ms (  1.22ms) 551.39us ( 78.77us)         7
  nlp_hess_l  |   2.00ms (399.60us)   2.00ms (399.25us)         5
   nlp_jac_g  | 101.00us ( 14.43us) 100.59us ( 14.37us)         7
       total  |  14.00ms ( 14.00ms)   6.00ms (  6.00ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999802642333, 0.43463954923507, 0.084989925087753]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 182.00us ( 30.33us) 181.97us ( 30.33us)         6
       nlp_g  |  49.00us (  8.17us)  49.11us (  8.19us)         6
    nlp_grad  | 104.00us (104.00

----------- MPC execution -----------
Predicted position of cube [0.49999433986174, 0.4363266717029, 0.084989998749382]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 239.00us ( 39.83us) 238.78us ( 39.80us)         6
       nlp_g  |  55.00us (  9.17us)  54.20us (  9.03us)         6
    nlp_grad  | 141.00us (141.00us) 141.19us (141.19us)         1
  nlp_grad_f  | 623.00us ( 89.00us) 624.11us ( 89.16us)         7
  nlp_hess_l  |  10.43ms (  2.09ms)   2.44ms (488.87us)         5
   nlp_jac_g  | 113.00us ( 16.14us) 113.61us ( 16.23us)         7
       total  |  22.97ms ( 22.97ms)   6.98ms (  6.98ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999434062647, 0.43632667169716, 0.084989999540188]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 179.00us ( 29.83us) 180.48us ( 30.08us)         6
       nlp_g  |  47.00us (  7.83us)  45.85us (  7.64us)         6
    nlp_grad  | 100.00us (100.00u

----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 190.00us ( 31.67us) 188.48us ( 31.41us)         6
       nlp_g  |  49.00us (  8.17us)  49.32us (  8.22us)         6
    nlp_grad  | 109.00us (109.00us) 109.99us (109.99us)         1
  nlp_grad_f  | 532.00us ( 76.00us) 536.22us ( 76.60us)         7
  nlp_hess_l  |   2.00ms (400.60us)   2.01ms (401.47us)         5
   nlp_jac_g  |  98.00us ( 14.00us)  98.65us ( 14.09us)         7
       total  |  13.79ms ( 13.79ms)   5.78ms (  5.78ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.22ms (203.33us) 242.97us ( 40.50us)         6
       nlp_g  | 363.00us ( 60.50us)  61.94us ( 10.32us)         6
    nlp_grad  | 133.00us (133.00

...

----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 187.00us ( 31.17us) 187.70us ( 31.28us)         6
       nlp_g  |  50.00us (  8.33us)  48.95us (  8.16us)         6
    nlp_grad  | 114.00us (114.00us) 114.45us (114.45us)         1
  nlp_grad_f  |   3.30ms (471.57us) 578.74us ( 82.68us)         7
  nlp_hess_l  |   6.34ms (  1.27ms)   2.08ms (416.57us)         5
   nlp_jac_g  | 104.00us ( 14.86us) 102.47us ( 14.64us)         7
       total  |  42.51ms ( 42.51ms)   6.14ms (  6.14ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 185.00us ( 30.83us) 184.89us ( 30.82us)         6
       nlp_g  |  46.00us (  7.67us)  47.53us (  7.92us)         6
    nlp_grad  | 110.00us (110.00

----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 198.00us ( 33.00us) 199.58us ( 33.26us)         6
       nlp_g  |  46.00us (  7.67us)  46.11us (  7.69us)         6
    nlp_grad  | 107.00us (107.00us) 107.88us (107.88us)         1
  nlp_grad_f  | 624.00us ( 89.14us) 625.30us ( 89.33us)         7
  nlp_hess_l  |  18.14ms (  3.63ms)   2.14ms (428.47us)         5
   nlp_jac_g  | 110.00us ( 15.71us) 110.49us ( 15.78us)         7
       total  |  22.29ms ( 22.29ms)   6.27ms (  6.27ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 211.00us ( 35.17us) 214.20us ( 35.70us)         6
       nlp_g  |  50.00us (  8.33us)  49.52us (  8.25us)         6
    nlp_grad  |  99.00us ( 99.00

----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 179.00us ( 29.83us) 179.84us ( 29.97us)         6
       nlp_g  |  44.00us (  7.33us)  44.02us (  7.34us)         6
    nlp_grad  | 111.00us (111.00us) 111.61us (111.61us)         1
  nlp_grad_f  | 543.00us ( 77.57us) 545.59us ( 77.94us)         7
  nlp_hess_l  |  10.01ms (  2.00ms)   2.03ms (406.63us)         5
   nlp_jac_g  |  98.00us ( 14.00us)  97.75us ( 13.96us)         7
       total  |  18.62ms ( 18.62ms)   5.77ms (  5.77ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 178.00us ( 29.67us) 177.31us ( 29.55us)         6
       nlp_g  |  44.00us (  7.33us)  42.99us (  7.17us)         6
    nlp_grad  | 105.00us (105.00

----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 888.00us (148.00us) 197.05us ( 32.84us)         6
       nlp_g  | 276.00us ( 46.00us)  54.12us (  9.02us)         6
    nlp_grad  | 112.00us (112.00us) 111.81us (111.81us)         1
  nlp_grad_f  |   3.58ms (511.29us) 615.87us ( 87.98us)         7
  nlp_hess_l  |  21.14ms (  4.23ms)   2.09ms (417.78us)         5
   nlp_jac_g  |   1.10ms (157.57us) 109.66us ( 15.67us)         7
       total  |  46.34ms ( 46.34ms)   6.39ms (  6.39ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 183.00us ( 30.50us) 183.27us ( 30.55us)         6
       nlp_g  |  42.00us (  7.00us)  43.30us (  7.22us)         6
    nlp_grad  | 116.00us (116.00

----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  | 585.00us ( 97.50us) 192.00us ( 32.00us)         6
       nlp_g  | 130.00us ( 21.67us)  56.09us (  9.35us)         6
    nlp_grad  | 516.00us (516.00us) 120.47us (120.47us)         1
  nlp_grad_f  |   1.46ms (208.29us) 612.96us ( 87.56us)         7
  nlp_hess_l  |  10.17ms (  2.03ms)   2.17ms (433.45us)         5
   nlp_jac_g  |   8.20ms (  1.17ms) 115.80us ( 16.54us)         7
       total  |  28.43ms ( 28.43ms)   6.72ms (  6.72ms)         1
----------- MPC execution -----------
Predicted position of cube [0.49999434107113, 0.43632667169387, 0.084989999999997]
      solver  :   t_proc      (avg)   t_wall      (avg)    n_eval
       nlp_f  |   1.14ms (190.50us) 183.95us ( 30.66us)         6
       nlp_g  |  47.00us (  7.83us)  45.74us (  7.62us)         6
    nlp_grad  | 112.00us (112.00

In [10]:
obj.end_simulation()

max(MPC_component.solver_times)

Ending simulation


0.016830921173095703